In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

df = pd.read_csv('us_perm_visas.csv', low_memory = False, parse_dates=['decision_date', 'case_received_date'])

### Checking the dimensions

In [2]:
print(df.shape)

(374362, 154)


### Merging the case number columns into a single column

In [3]:
casenoloc = df.columns.get_loc("case_no")
casenumberloc = df.columns.get_loc("case_number")
newcasenumberlist = []

for casenumber in df.iloc[0:135269,casenoloc]:
    newcasenumberlist.append(casenumber)
    
for casenumber in df.iloc[135269:374363,casenumberloc]:
    newcasenumberlist.append(casenumber)
    
df['casenumber'] = newcasenumberlist
df.drop(df.columns[[casenoloc,casenumberloc]], axis=1, inplace=True)

### Simplifying the target column

In [4]:
df = df[df["case_status"] != 'Withdrawn']

df.loc[df.case_status == 'Certified-Expired', 'case_status'] = 'Certified'
df.case_status.value_counts()

Certified    330519
Denied        25649
Name: case_status, dtype: int64

### Looking at the year only

In [5]:
df['year'] = df['decision_date'].dt.year

### Converting the salaries to a standard unit ($/year)

In [7]:
#Replacing commas with whitespace character
df['pw_amount_9089'] = df['pw_amount_9089'].str.replace(",","") 

for unit in df.pw_unit_of_pay_9089.unique():
    if unit == "hr" or unit == "Hour":
        df.loc[df['pw_unit_of_pay_9089'] == unit, 'pw_amount_9089'] = df['pw_amount_9089'].apply(lambda x: float(x) * 8 * 250)
        df.loc[df['pw_unit_of_pay_9089'] == unit, 'pw_unit_of_pay_9089'] = df['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year") 
    elif unit == "wk" or unit == "Week":
        df.loc[df['pw_unit_of_pay_9089'] == unit, 'pw_amount_9089'] = df['pw_amount_9089'].apply(lambda x: float(x) * 50)
        df.loc[df['pw_unit_of_pay_9089'] == unit, 'pw_unit_of_pay_9089'] = df['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    elif unit == "mth" or unit == "Month":
        df.loc[df['pw_unit_of_pay_9089'] == unit, 'pw_amount_9089'] = df['pw_amount_9089'].apply(lambda x: float(x) * 12)
        df.loc[df['pw_unit_of_pay_9089'] == unit, 'pw_unit_of_pay_9089'] = df['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    elif unit == "bi" or unit == "Bi-Weekly":  
        df.loc[df['pw_unit_of_pay_9089'] == unit, 'pw_amount_9089'] = df['pw_amount_9089'].apply(lambda x: float(x) * 25)
        df.loc[df['pw_unit_of_pay_9089'] == unit, 'pw_unit_of_pay_9089'] = df['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    elif unit =="yr":
         df.loc[df['pw_unit_of_pay_9089'] == unit ,'pw_unit_of_pay_9089'] = df['pw_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    else:
        continue
        
#Filling missing values with median 
df['pw_amount_9089']= df['pw_amount_9089'].fillna((df['pw_amount_9089'].median()))

#Changing format from string to float
df['pw_amount_9089'] = df.pw_amount_9089.astype(float)

#Displaying 10 first values
df[['pw_amount_9089','pw_unit_of_pay_9089']].head(10)

,pw_amount_9089,pw_unit_of_pay_9089
0,75629.00,Year
1,37024.00,Year
2,47923.00,Year
3,21940.00,Year
4,94890.00,Year
5,37024.00,Year
6,47083.33,Year
7,36733.00,Year
8,44824.00,Year
9,25720.00,Year


### Checking which columns have  <15% null values

In [8]:
percent_nonnull = 85 
nonnullcount = 374362 * percent_nonnull/100
df = df.loc[:,df.count() >= nonnullcount]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356168 entries, 0 to 374353
Data columns (total 19 columns):
case_status               356168 non-null object
class_of_admission        335083 non-null object
country_of_citizenship    336896 non-null object
decision_date             356168 non-null datetime64[ns]
employer_address_1        356131 non-null object
employer_city             356158 non-null object
employer_name             356160 non-null object
employer_postal_code      356135 non-null object
employer_state            356131 non-null object
job_info_work_city        356073 non-null object
job_info_work_state       356072 non-null object
pw_amount_9089            356168 non-null float64
pw_level_9089             329583 non-null object
pw_soc_code               355778 non-null object
pw_soc_title              353847 non-null object
pw_source_name_9089       354081 non-null object
pw_unit_of_pay_9089       354687 non-null object
casenumber                356168 non-null objec

### Selecting features

In [35]:
df.iloc[0,[0,1,2,5,6,8,11,14,15,18]]

case_status                                               Certified
class_of_admission                                              J-1
country_of_citizenship                                        INDIA
employer_city                                              NEW YORK
employer_name                                      NETSOFT USA INC.
employer_state                                                   NY
pw_amount_9089                                                75629
pw_soc_title              Computer Software Engineers, Applications
pw_source_name_9089                                             OES
year                                                           2012
Name: 0, dtype: object

### Imputing values to replace nulls

In [17]:
df['class_of_admission']=df['class_of_admission'].fillna((df['class_of_admission'].mode()[0]))

In [18]:
df['country_of_citizenship']=df['country_of_citizenship'].fillna((df['country_of_citizenship'].mode()[0]))

In [22]:
df['employer_city']=df['employer_city'].fillna((df['employer_city'].mode()[0]))
df['employer_name']=df['employer_name'].fillna((df['employer_name'].mode()[0]))
df['employer_name']=df['employer_name'].astype(str).str.upper()
df['pw_source_name_9089']=df['pw_source_name_9089'].fillna((df['pw_source_name_9089'].mode()[0]))
df['employer_state'] = df['employer_state'].fillna(df['employer_state'].mode()[0])

In [39]:
df_selected = df.iloc[:,[0,1,2,5,6,8,11,14,15,18]]

In [40]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356168 entries, 0 to 374353
Data columns (total 10 columns):
case_status               356168 non-null object
class_of_admission        356168 non-null object
country_of_citizenship    356168 non-null object
employer_city             356168 non-null object
employer_name             356168 non-null object
employer_state            356168 non-null object
pw_amount_9089            356168 non-null float64
pw_soc_title              353847 non-null object
pw_source_name_9089       356168 non-null object
year                      356168 non-null int64
dtypes: float64(1), int64(1), object(8)
memory usage: 39.9+ MB
